In [1]:
import numpy as np
import pandas as pd

from snmp_fetch.pandas_extension import xt
from snmp_fetch.pandas_extension.utils import column_names

df = pd.DataFrame(
    data=[[x] for x in np.arange(0, 1000000).reshape(1000, 1000)],
    columns=['A'])
df

,A
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[1000, 1001, 1002, 1003, 1004, 1005, 1006, 100..."
2,"[2000, 2001, 2002, 2003, 2004, 2005, 2006, 200..."
3,"[3000, 3001, 3002, 3003, 3004, 3005, 3006, 300..."
4,"[4000, 4001, 4002, 4003, 4004, 4005, 4006, 400..."
...,...
995,"[995000, 995001, 995002, 995003, 995004, 99500..."
996,"[996000, 996001, 996002, 996003, 996004, 99600..."
997,"[997000, 997001, 997002, 997003, 997004, 99700..."
998,"[998000, 998001, 998002, 998003, 998004, 99800..."


In [2]:
arr = np.array(df['A'].values.tolist())
pd.DataFrame(
dict(zip(column_names(2), [arr[:, s].tolist() for s in [0, slice(None, 5)]]))
)

,A,B
0,0,"[0, 1, 2, 3, 4]"
1,10,"[10, 11, 12, 13, 14]"
2,20,"[20, 21, 22, 23, 24]"
3,30,"[30, 31, 32, 33, 34]"
4,40,"[40, 41, 42, 43, 44]"
5,50,"[50, 51, 52, 53, 54]"
6,60,"[60, 61, 62, 63, 64]"
7,70,"[70, 71, 72, 73, 74]"
8,80,"[80, 81, 82, 83, 84]"
9,90,"[90, 91, 92, 93, 94]"


In [172]:
%%timeit
df['A'].xt[-28, -106, 470, -470]

2.12 ms ± 249 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [2]:
%%timeit
df['A'].xt[0, :20]

6.24 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [173]:
df['A'].xt[-28, -106, 470, -470]

,A,B
0,972,894
1,1972,1894
2,2972,2894
3,3972,3894
4,4972,4894
...,...,...
995,995972,995894
996,996972,996894
997,997972,997894
998,998972,998894


In [5]:
df = pd.DataFrame([[np.array([0, 1])],[np.array([2, 3])],], columns=['A'])

In [6]:
df['A'].xt[0].dtypes

dtype('int64')

In [7]:
pd.DataFrame([[0, [1]], [2, [3]]]).equals(df['A'].xt[0, 1:2])

False

In [8]:
pd.DataFrame([[0, [1]], [2, [3]]], columns=['A', 'B']).eq(df['A'].xt[0, 1:2])

,A,B
0,True,False
1,True,False


In [7]:
df0 = df['A'].xt[0, 500:]
df0 = df0.set_index('A')
df0[['a', 'b']] = df0['B'].xt[2, :250]
df0

,B,a,b
A,,,
0,"[500, 501, 502, 503, 504, 505, 506, 507, 508, ...",502,"[500, 501, 502, 503, 504, 505, 506, 507, 508, ..."
1000,"[1500, 1501, 1502, 1503, 1504, 1505, 1506, 150...",1502,"[1500, 1501, 1502, 1503, 1504, 1505, 1506, 150..."
2000,"[2500, 2501, 2502, 2503, 2504, 2505, 2506, 250...",2502,"[2500, 2501, 2502, 2503, 2504, 2505, 2506, 250..."
3000,"[3500, 3501, 3502, 3503, 3504, 3505, 3506, 350...",3502,"[3500, 3501, 3502, 3503, 3504, 3505, 3506, 350..."
4000,"[4500, 4501, 4502, 4503, 4504, 4505, 4506, 450...",4502,"[4500, 4501, 4502, 4503, 4504, 4505, 4506, 450..."
...,...,...,...
995000,"[995500, 995501, 995502, 995503, 995504, 99550...",995502,"[995500, 995501, 995502, 995503, 995504, 99550..."
996000,"[996500, 996501, 996502, 996503, 996504, 99650...",996502,"[996500, 996501, 996502, 996503, 996504, 99650..."
997000,"[997500, 997501, 997502, 997503, 997504, 99750...",997502,"[997500, 997501, 997502, 997503, 997504, 99750..."


In [76]:
from hypothesis.extra.numpy import arrays, from_dtype, basic_indices
import hypothesis.strategies as st

In [98]:
arr=arrays(
    dtype=st.one_of(from_dtype(np.dtype(np.uint8)), from_dtype(np.dtype(np.uint64))),
    shape=st.tuples(
        st.integers(min_value=0, max_value=1000),
        st.integers(min_value=0, max_value=1000)
    )
).example()
df = pd.DataFrame({'A': [a for a in arr]})
result = df['A'].xt[0]

result.equals(pd.Series([a[0] for a in arr]).rename('A'))

False

[slice(954, None, -1)]

In [2]:
column = 'A'
array = np.array([[0, 0]], dtype=np.uint8)
indicies = [0, slice(0, 2, 1)]
df = pd.DataFrame({column: [a for a in array]})
df

,A
0,"[0, 0]"


In [3]:
can = pd.concat(
    [
        df[column].apply(lambda x: x[i]).astype(array.dtype.type).rename(n)
        if isinstance(i, int) else
        df[column].apply(lambda x: x[i]).rename(n)
        for n, i in zip(column_names(len(indicies)), indicies)
    ],
    axis=1
)
can

In [20]:
can.dtypes

A     uint8
B    object
dtype: object

In [12]:
df2 = df[column].xt[indicies]
df2

,A,B
0,0,"[0, 0]"


In [19]:
df2.dtypes

A     uint8
B    object
dtype: object

In [15]:
can['B'][0]

array([0, 0], dtype=uint8)

In [61]:
df2.eq(can)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [56]:
df2

,A,B
0,0,"[0, 0]"


In [57]:
can

,A,B
0,0,"[0, 0]"


In [89]:
for (a, b) in zip(df2, can):
    if isinstance(df2[a][0], np.ndarray):
        for (c, d) in zip(df2[a], can[b]):
            print(np.array_equal(c, d))
    else:
        print(df2[a] == can[b])

0    True
Name: A, dtype: bool
True


In [83]:
np.array_equal(df2['B'][0], can['B'][0])

True

array([0, 0], dtype=uint8)